# Agenda

1. Iterators and the iterator protocol
    - The protocol itself
    - Making our classes iterable
    - Generators and generator functions
    - Generator expressions / generator comprehensions
    - `itertools`
2. Decorators
3. Concurrency
    - Threads
    - Multiprocessing
    - `asyncio`


# Iterators

We know that we can put many different data types into a `for` loop:

- Strings, and we get one character per iteration
- Lists or tuples, one element per iteration
- Dicts, one key per iteration
- Files, one line per iteration



In [1]:
for one_character in 'abcd':
    print(one_character)

a
b
c
d


# Iterator protocol

1. `for` turns to the object at the end of the line, and asks: Are you iterable?
    - If not, then we exit with a `TypeError` exception
2. If so, then `for` says: Give me your next value
    - If there are no more values, then the loop exits
3. The next value is assigned to the loop variable (`one_character`)
4. The loop body executes with that assignment
5. We return